In [ ]:
import pandas as pd
from datetime import datetime

# Define required fields (matching app.py)
REQUIRED_FIELDS = [
    'deal_id', 'advertiser_name', 'io_name', 'product_name', 'buyer_seat_id',
    'deal_list_id', 'market_country', 'platform', 'device_types', 'cpm',
    'flight_start', 'flight_end', 'creative_type', 'inventory_type', 'status'
]

VALID_STATUSES = ['Active', 'Inactive', 'Paused', 'Archived']
VALID_PLATFORMS = ['Web', 'App', 'Both']
VALID_INVENTORY_TYPES = ['Direct', 'Open Market', 'Private Auction']

class DealAuditor:
    """Simplified Deal Audit with Data Completeness Focus"""
    
    def __init__(self):
        self.validation_results = {}
        
    def validate_deal(self, deal_data):
        """Run comprehensive audit on deal"""
        results = {}
        total_checks = 0
        checks_passed = 0
        
        # Check required fields
        results['required_fields'] = self._check_required_fields(deal_data)
        if results['required_fields']['passed']: checks_passed += 1
        total_checks += 1
        
        # Check deal ID
        results['deal_id'] = self._validate_field(deal_data.get('deal_id'), 'Deal ID')
        if results['deal_id']['passed']: checks_passed += 1
        total_checks += 1
        
        # Check advertiser
        results['advertiser'] = self._validate_field(deal_data.get('advertiser_name'), 'Advertiser Name')
        if results['advertiser']['passed']: checks_passed += 1
        total_checks += 1
        
        # Check buyer seat
        results['buyer_seat'] = self._validate_field(deal_data.get('buyer_seat_id'), 'Buyer Seat ID')
        if results['buyer_seat']['passed']: checks_passed += 1
        total_checks += 1
        
        # Check deal list
        results['deal_list'] = self._validate_field(deal_data.get('deal_list_id'), 'Deal List ID')
        if results['deal_list']['passed']: checks_passed += 1
        total_checks += 1
        
        # Check targeting
        results['targeting'] = self._validate_targeting(deal_data)
        if results['targeting']['passed']: checks_passed += 1
        total_checks += 1
        
        # Check CPM
        results['cpm'] = self._validate_cpm(deal_data.get('cpm'))
        if results['cpm']['passed']: checks_passed += 1
        total_checks += 1
        
        # Check flight dates
        results['flight_dates'] = self._validate_flight_dates(deal_data.get('flight_start'), deal_data.get('flight_end'))
        if results['flight_dates']['passed']: checks_passed += 1
        total_checks += 1
        
        # Check creative
        results['creative'] = self._validate_creative(deal_data)
        if results['creative']['passed']: checks_passed += 1
        total_checks += 1
        
        # Check status
        results['status'] = self._validate_field(deal_data.get('status'), 'Status', VALID_STATUSES)
        if results['status']['passed']: checks_passed += 1
        total_checks += 1
        
        results['overall_outcome'] = self._calculate_outcome(checks_passed, total_checks)
        return results
    
    def _check_required_fields(self, deal_data):
        missing = [f for f in REQUIRED_FIELDS if not deal_data.get(f)]
        passed = len(missing) == 0
        status = f'{len(REQUIRED_FIELDS) - len(missing)}/{len(REQUIRED_FIELDS)} present'
        issue = f'Missing: {", ".join(missing)}' if missing else None
        return {'passed': passed, 'status': status, 'issue': issue}
    
    def _validate_field(self, value, field_name, valid_values=None):
        if not value or (isinstance(value, str) and value.strip() == ''):
            return {'passed': False, 'status': 'Missing', 'issue': f'{field_name} is required'}
        
        if valid_values and value not in valid_values:
            return {'passed': False, 'status': 'Invalid', 'issue': f'Must be: {", ".join(valid_values)}'}
        
        return {'passed': True, 'status': 'Valid', 'issue': None}
    
    def _validate_targeting(self, deal_data):
        issues = []
        if not deal_data.get('market_country'): issues.append('Market/Country missing')
        if not deal_data.get('platform'): issues.append('Platform missing')
        if deal_data.get('platform') not in VALID_PLATFORMS: issues.append('Invalid platform')
        if not deal_data.get('device_types'): issues.append('Device types missing')
        
        passed = len(issues) == 0
        return {'passed': passed, 'status': 'Complete' if passed else 'Incomplete', 'issue': '; '.join(issues) if issues else None}
    
    def _validate_cpm(self, cpm):
        if not cpm: return {'passed': False, 'status': 'Missing', 'issue': 'CPM not specified'}
        try:
            cpm_val = float(cpm)
            if cpm_val <= 0: return {'passed': False, 'status': 'Invalid', 'issue': 'CPM must be > 0'}
            return {'passed': True, 'status': f'${round(cpm_val, 2)}', 'issue': None}
        except:
            return {'passed': False, 'status': 'Invalid', 'issue': 'CPM must be a valid number'}
    
    def _validate_flight_dates(self, start, end):
        issues = []
        if not start: issues.append('Flight start missing')
        if not end: issues.append('Flight end missing')
        passed = len(issues) == 0
        return {'passed': passed, 'status': 'Valid' if passed else 'Invalid', 'issue': '; '.join(issues) if issues else None}
    
    def _validate_creative(self, deal_data):
        issues = []
        if not deal_data.get('creative_type'): issues.append('Creative type missing')
        if not deal_data.get('creative_sizes'): issues.append('Creative sizes missing')
        passed = len(issues) == 0
        return {'passed': passed, 'status': 'Complete' if passed else 'Incomplete', 'issue': '; '.join(issues) if issues else None}
    
    def _calculate_outcome(self, checks_passed, total_checks):
        percentage = (checks_passed / total_checks) * 100
        if percentage >= 90:
            return {'outcome': 'READY ✅', 'color': 'green', 'percentage': percentage, 'recommendation': 'Deal ready for activation'}
        elif percentage >= 70:
            return {'outcome': 'REVIEW ⚠️', 'color': 'orange', 'percentage': percentage, 'recommendation': 'Review flagged fields'}
        else:
            return {'outcome': 'INCOMPLETE ❌', 'color': 'red', 'percentage': percentage, 'recommendation': 'Complete required fields'}

# Example usage
print("Deal ID Audit Tool - Sample Execution\n")
print("=" * 60)

# Sample deal with ALL required columns
sample_deal = {
    'deal_id': 'D-2024-001',
    'advertiser_name': 'TechCorp Inc',
    'io_name': 'Q1-2024-Digital-Campaign',
    'product_name': 'Banner Display',
    'buyer_seat_id': 'BS-45678',
    'publisher_targeting': 'Premium Publishers',
    'deal_list_id': 'DL-98765',
    'market_country': 'US, CA',
    'platform': 'Web',
    'device_types': 'Desktop, Mobile',
    'ad_sizes': '300x250, 728x90',
    'kvps': 'brand=tech;inventory=display',
    'auction_type': 'Private Auction',
    'cpm': 8.50,
    'priority': 'High',
    'frequency_cap': '5 per user/day',
    'budget': 50000,
    'impression_goal': 1000000,
    'flight_start': '2024-01-01',
    'flight_end': '2024-03-31',
    'creative_type': 'Static Display',
    'creative_sizes': '300x250, 728x90',
    'inventory_type': 'Direct',
    'status': 'Active',
    'comments': 'High-priority Q1 campaign',
    'monetize_280': 'Yes',
    'curate_seat': 'CS-1234'
}

# Run audit
auditor = DealAuditor()
results = auditor.validate_deal(sample_deal)

print(f"Deal ID: {sample_deal['deal_id']}")
print(f"Advertiser: {sample_deal['advertiser_name']}")
print(f"Status: {sample_deal['status']}")
print("\n" + "=" * 60)
print("AUDIT RESULTS")
print("=" * 60)

outcome = results['overall_outcome']
print(f"\n🎯 Overall Outcome: {outcome['outcome']}")
print(f"📊 Completion: {outcome['percentage']:.1f}%")
print(f"💡 Recommendation: {outcome['recommendation']}")

print("\n" + "=" * 60)
print("INDIVIDUAL CHECKS")
print("=" * 60)

check_order = ['required_fields', 'deal_id', 'advertiser', 'buyer_seat', 'deal_list', 'targeting', 'cpm', 'flight_dates', 'creative', 'status']

for check_name in check_order:
    if check_name in results:
        result = results[check_name]
        icon = "✅" if result['passed'] else "❌"
        print(f"\n{icon} {check_name.replace('_', ' ').title()}")
        print(f"   Status: {result['status']}")
        if result['issue']:
            print(f"   Issue: {result['issue']}")

print("\n" + "=" * 60)

: 

In [ ]:
# 🟢 Deal ID Audit Tool - Streamlit Application

## Quick Links

### 📖 Documentation
- **[START_HERE.md](file:///c%3A/Users/pastormarta/Vscode/START_HERE.md)** - Quick start guide
- **[INSTALLATION_GUIDE.txt](file:///c%3A/Users/pastormarta/Vscode/INSTALLATION_GUIDE.txt)** - Step-by-step setup
- **[STREAMLIT_GUIDE.md](file:///c%3A/Users/pastormarta/Vscode/STREAMLIT_GUIDE.md)** - Complete user manual
- **[ARCHITECTURE.md](file:///c%3A/Users/pastormarta/Vscode/ARCHITECTURE.md)** - Technical details
- **[PROJECT_COMPLETE.txt](file:///c%3A/Users/pastormarta/Vscode/PROJECT_COMPLETE.txt)** - Project summary

### 💻 Application
- **[app.py](file:///c%3A/Users/pastormarta/Vscode/app.py)** - Main Streamlit application
- **[requirements.txt](file:///c%3A/Users/pastormarta/Vscode/requirements.txt)** - Python dependencies
- **[sample_deals.csv](file:///c%3A/Users/pastormarta/Vscode/sample_deals.csv)** - Example data

### 📚 Full Index
- **[INDEX.md](file:///c%3A/Users/pastormarta/Vscode/INDEX.md)** - Complete file index

---

## Quick Start

```bash
# 1. Install dependencies
pip install -r requirements.txt

# 2. Run application
streamlit run app.py

# 3. Open browser
# http://localhost:8501
```

---

## Project Structure

```
c:\Users\pastormarta\Vscode\
├── app.py ⭐ (Main application)
├── requirements.txt (Dependencies)
├── setup.ps1 (Setup script)
├── setup.bat (Setup script)
│
├── Documentation/
│   ├── START_HERE.md
│   ├── INSTALLATION_GUIDE.txt
│   ├── STREAMLIT_GUIDE.md
│   ├── ARCHITECTURE.md
│   ├── CHECKLIST.md
│   ├── PROJECT_COMPLETE.txt
│   └── INDEX.md
│
├── Data/
│   └── sample_deals.csv
│
└── Notebooks/
    └── Untitled-1.ipynb (This file)
```